In [66]:
import matplotlib.pyplot as plt
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

In [67]:
file = h5py.File("./dataset.h5")

In [68]:
#This is the full function for loading this data
def load():
    f = h5py.File("./dataset.h5")
    x = f['x'].value
    y = f['y'].value
    f.close()
    
    x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=100)
    
    # Making the data channel last
    # 部分圖片的儲存是 (channel ,[height][width] ) 的方法儲存，為了修正，需要調用rollaxis
    x_train = np.rollaxis(x_train, 1, 4)
    x_test = np.rollaxis(x_test, 1, 4)
    
    # Normalizing data
    x_train = x_train  / 255.0
    x_test = x_test / 255.0
   
    return x_train, x_test, y_train, y_test

In [69]:
x_train, x_test, y_train, y_test = load()

c:\users\shengwei\appdata\local\programs\python\python36\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [70]:
num_classes = len(y_test[0])

In [71]:
#for 1 neural softmax
#y_test_List = [ y_test[:,i].reshape(len(y_test),1) for i in range(num_classes)]
#y_train_List = [ y_train[:,i].reshape(len(y_train),1) for i in range(num_classes)]

In [72]:
#for 2 neural softmax
y_test_List = []
for i in range(num_classes):
    new_list = []
    for ele in y_test[:,i] :
        other_class = 0 if ele == 1 else 1
        new_ele = [ele,other_class]
        new_list.append(new_ele)
    y_test_List.append(np.array(new_list))
y_train_List = []
for i in range(num_classes):
    new_list = []
    for ele in y_train[:,i] :
        other_class = 0 if ele == 1 else 1
        new_ele = [ele,other_class]
        new_list.append(new_ele)
    y_train_List.append(np.array(new_list))

In [73]:
y_test.shape

(400, 5)

In [74]:
y_test_List[0].shape

(400, 2)

In [75]:
type(y_test_List)

list

In [76]:
y_test_List[0].shape

(400, 2)

In [77]:
y_train_List[0].shape

(1600, 2)

In [78]:
import keras.backend as K
def multitask_loss(y_true, y_pred):
# Avoid divide by 0
    #keras.backend.clip(x, min_value, max_value)
        # x is a tensor
        # this function will return a tensor
        # clip will made the value in tensor within min and max value
        # use it with epsilon() can help make the value within 0~1 but not 0, so it can avoid divide by 0
    #keras.backend.epsilon()
        #return a float which is very small 
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
# Multi-task loss
    # the formule shows below
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

In [79]:
from keras.models import Sequential # for buliding the model
from keras.layers import Dense, Dropout, Flatten, Input# for fullconnection layers
from keras.layers import Conv2D, MaxPooling2D, Activation # for CNN
from keras.models import Model
import keras.backend as K

img_rows, img_cols = 100, 100
channels = 3

In [80]:
#share layers
In_layer = Input(shape=(img_rows, img_cols, channels))

C_L_1_1 = Conv2D(32, kernel_size=(3, 3),padding='same',activation = 'relu')(In_layer)
C_L_1_2 = Conv2D(32, kernel_size=(3, 3),padding='same',activation = 'relu')(C_L_1_1)
MP_L_1 = MaxPooling2D(pool_size=(2, 2))(C_L_1_2)
DO_L_1 = Dropout(0.25)(MP_L_1)

C_L_2_1 = Conv2D(32, kernel_size=(3, 3),padding='same',activation = 'relu')(DO_L_1)
C_L_2_2 = Conv2D(32, kernel_size=(3, 3),padding='same',activation = 'relu')(C_L_2_1)
MP_L_2 = MaxPooling2D(pool_size=(2, 2))(C_L_2_2)
DO_L_2 = Dropout(0.25)(MP_L_2)

Flatten_L = Flatten()(DO_L_2)

In [81]:
#multi_task
Output_Layers = []
for i in range(num_classes):
    MTL_L = Dense(2, activation='sigmoid', name='Tag_'+str(i))(Flatten_L)
    Output_Layers.append(MTL_L)

In [82]:
model = Model(inputs=In_layer, outputs=Output_Layers)

In [83]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100, 100, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 100, 100, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 100, 100, 32) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 50, 50, 32)   0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
dropout_1 

In [94]:
model.compile(optimizer='rmsprop', 
              loss= "binary_crossentropy",
              loss_weights=[1., 1., 1., 1., 1.],
              metrics=['accuracy'])

In [95]:
batch_size = 100
epochs = 10

In [96]:
#this auc fumction works
#add this in model.fit()  :callbacks=callbacks
from keras.callbacks import  Callback
from sklearn.metrics import  roc_auc_score
class roc_auc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict_proba(self.x, verbose=0)
        roc = roc_auc_score(self.y, y_pred)
        logs['roc_auc'] = roc_auc_score(self.y, y_pred)
        logs['norm_gini'] = ( roc_auc_score(self.y, y_pred) * 2 ) - 1

        y_pred_val = self.model.predict_proba(self.x_val, verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        logs['roc_auc_val'] = roc_auc_score(self.y_val, y_pred_val)
        logs['norm_gini_val'] = ( roc_auc_score(self.y_val, y_pred_val) * 2 ) - 1

        print('\rroc_auc: %s - roc_auc_val: %s - norm_gini: %s - norm_gini_val: %s' % (str(round(roc,5)),str(round(roc_val,5)),str(round((roc*2-1),5)),str(round((roc_val*2-1),5))), end=10*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return
callbacks = [roc_auc_callback(training_data=(x_train, [y for y in y_train_List]),validation_data=(x_test, [y for y in y_test_List]))]

In [97]:
model.fit(x_train, [y for y in y_train_List],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          #callbacks=callbacks,
          validation_data=(x_test, [y for y in y_test_List]))

Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 58s 36ms/step - loss: 1.7573 - Tag_0_loss: 0.2542 - Tag_1_loss: 0.3312 - Tag_2_loss: 0.4957 - Tag_3_loss: 0.1904 - Tag_4_loss: 0.4858 - Tag_0_acc: 0.9028 - Tag_1_acc: 0.8772 - Tag_2_acc: 0.7856 - Tag_3_acc: 0.9303 - Tag_4_acc: 0.8444 - val_loss: 1.7157 - val_Tag_0_loss: 0.3100 - val_Tag_1_loss: 0.4060 - val_Tag_2_loss: 0.4212 - val_Tag_3_loss: 0.1976 - val_Tag_4_loss: 0.3809 - val_Tag_0_acc: 0.8825 - val_Tag_1_acc: 0.8325 - val_Tag_2_acc: 0.8037 - val_Tag_3_acc: 0.9325 - val_Tag_4_acc: 0.8575
Epoch 2/10
1600/1600 [==============================] - 59s 37ms/step - loss: 1.1932 - Tag_0_loss: 0.1972 - Tag_1_loss: 0.2577 - Tag_2_loss: 0.3794 - Tag_3_loss: 0.1278 - Tag_4_loss: 0.2311 - Tag_0_acc: 0.9237 - Tag_1_acc: 0.8909 - Tag_2_acc: 0.8241 - Tag_3_acc: 0.9534 - Tag_4_acc: 0.9097 - val_loss: 1.7700 - val_Tag_0_loss: 0.2896 - val_Tag_1_loss: 0.4138 - val_Tag_2_loss: 0.4225 - val_Tag_3_los

In [87]:
model.fit(x_train, [y for y in y_train_List],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          #callbacks=callbacks,
          validation_data=(x_test, [y for y in y_test_List]))

Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 62s 39ms/step - loss: 6.4126 - Tag_0_loss: 1.2244 - Tag_1_loss: 1.2294 - Tag_2_loss: 1.5120 - Tag_3_loss: 1.1373 - Tag_4_loss: 1.3094 - Tag_0_acc: 0.7681 - Tag_1_acc: 0.7662 - Tag_2_acc: 0.7012 - Tag_3_acc: 0.7713 - Tag_4_acc: 0.7237 - val_loss: 5.5938 - val_Tag_0_loss: 1.0995 - val_Tag_1_loss: 1.0082 - val_Tag_2_loss: 1.0643 - val_Tag_3_loss: 0.8160 - val_Tag_4_loss: 1.6058 - val_Tag_0_acc: 0.7900 - val_Tag_1_acc: 0.7625 - val_Tag_2_acc: 0.7425 - val_Tag_3_acc: 0.8000 - val_Tag_4_acc: 0.4650
Epoch 2/10
1600/1600 [==============================] - 64s 40ms/step - loss: 4.8695 - Tag_0_loss: 0.8994 - Tag_1_loss: 1.0026 - Tag_2_loss: 1.1562 - Tag_3_loss: 0.7299 - Tag_4_loss: 1.0814 - Tag_0_acc: 0.8100 - Tag_1_acc: 0.7444 - Tag_2_acc: 0.7044 - Tag_3_acc: 0.8469 - Tag_4_acc: 0.7650 - val_loss: 5.1022 - val_Tag_0_loss: 0.8619 - val_Tag_1_loss: 1.1012 - val_Tag_2_loss: 1.2558 - val_Tag_3_los

In [ ]:
#single neural
val_Tag_0_acc: 0.8825 - val_Tag_1_acc: 0.8350 - val_Tag_2_acc: 0.7975 - val_Tag_3_acc: 0.8975 - val_Tag_4_acc: 0.8475
#double neural
val_Tag_0_acc: 0.8787 - val_Tag_1_acc: 0.8237 - val_Tag_2_acc: 0.7587 - val_Tag_3_acc: 0.8875 - val_Tag_4_acc: 0.8325

In [44]:
#single neural softmax
(0.8825+0.8350+0.7975+0.8975+0.8475)/5

0.852

In [45]:
#double neural
(0.8787+0.8237+0.7587+0.8875+0.8325)/5

0.8362200000000002

In [89]:
#double neural with multitask loss
(0.8925+0.8350+0.8050+0.9125+0.8625)/5

0.8615

In [92]:
(0.8950+0.8275+0.7375+0.9150+0.8275)/5

0.8404999999999999

In [93]:
model.predict(np.array([x_train[0]]))

[array([[0.01945499, 0.8940216 ]], dtype=float32),
 array([[0.01275013, 0.99992716]], dtype=float32),
 array([[0.74234015, 0.8995152 ]], dtype=float32),
 array([[0.90993214, 0.11541927]], dtype=float32),
 array([[0.06273694, 0.97387433]], dtype=float32)]

In [63]:
def func(x):    
    a = x.eval(session=tf.Session())
    a[x < 0.5] = 0
    a[x >= 0.5] = 1
    return K.variable(a)

AttributeError: 'float' object has no attribute 'eval'

In [54]:
 K.greater(0.01,0.5)

<tf.Tensor 'Greater_8:0' shape=() dtype=bool>

In [58]:
K.sigmoid(0.1)

<tf.Tensor 'Sigmoid_2:0' shape=() dtype=float32>